In [1]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 906.1 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.7/308.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.5 MB/s eta

In [10]:
!pip install langchain_openai

In [4]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from textwrap import dedent
# Set environment variables
os.environ["OPENAI_API_KEY"] = ""

# Define the OpenAI models
llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 1. The Expert Pattern

In [ ]:
# Create expert agents
programming_expert = Agent(
    role="Programming Expert",
    goal="Provide expert Python programming solutions",
    backstory=dedent("""
        You are a senior Python developer with 15 years of experience.
        You have extensive knowledge of Python libraries, best practices, and
        efficient coding patterns. You excel at explaining complex programming
        concepts in a clear and accessible way.
    """),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

math_expert = Agent(
    role="Mathematics Expert",
    goal="Solve mathematical problems with clear step-by-step explanations",
    backstory=dedent("""
        You have a Ph.D. in Applied Mathematics and have taught at prestigious
        universities. You specialize in calculus, statistics, and linear algebra.
        You are known for your ability to break down complex mathematical concepts
        into understandable steps.
    """),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

physics_expert = Agent(
    role="Physics Expert",
    goal="Explain physics concepts and solve physics problems",
    backstory=dedent("""
        You are a theoretical physicist with expertise in mechanics and electromagnetism.
        You have published numerous papers in top journals and have a talent for explaining
        physics using intuitive analogies and clear mathematical formulations.
    """),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

coordinator = Agent(
    role="Team Coordinator",
    goal="Coordinate the expert team and synthesize their inputs",
    backstory=dedent("""
        You are an experienced project manager who specializes in coordinating
        multidisciplinary teams. You have a talent for identifying which expert
        is best suited for a particular question and synthesizing diverse inputs
        into coherent final answers.
    """),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

# Define tasks
def create_tasks(question):
    assessment_task = Task(
        description=f"Analyze this question and determine which experts need to be consulted: '{question}'",
        expected_output="A list of experts needed and why they're needed for this specific question",
        agent=coordinator
    )

    programming_task = Task(
        description=f"Provide a programming solution for this aspect of the question: '{question}'",
        expected_output="Python code with detailed explanations",
        agent=programming_expert,
        context=[assessment_task]
    )

    math_task = Task(
        description=f"Solve the mathematical aspects of this question: '{question}'",
        expected_output="Step-by-step mathematical solution",
        agent=math_expert,
        context=[assessment_task]
    )

    physics_task = Task(
        description=f"Explain the physics concepts relevant to this question: '{question}'",
        expected_output="Clear explanation of physics principles and formulas",
        agent=physics_expert,
        context=[assessment_task]
    )

    synthesis_task = Task(
        description="Synthesize all expert inputs into a comprehensive final answer",
        expected_output="A complete answer that incorporates all relevant expert knowledge",
        agent=coordinator,
        context=[programming_task, math_task, physics_task]
    )

    return [assessment_task, programming_task, math_task, physics_task, synthesis_task]

# Create crew
def solve_problem_with_experts(question):
    # Create tasks for this specific question
    tasks = create_tasks(question)

    # Create the crew
    expert_crew = Crew(
    agents=[coordinator, programming_expert, math_expert, physics_expert],
    tasks=tasks,
    verbose=True,  # Changed from 2 to True
    process=Process.sequential
)

    # Get the crew to work!
    result = expert_crew.kickoff()

    return result

# Example usage
if __name__ == "__main__":
    question = "I need to calculate the volume of a sphere with radius r=5cm and then write a Python function to calculate this for any radius."
    result = solve_problem_with_experts(question)
    print("\nFinal Answer:")
    print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 34b9e3ae-c453-41a9-a1fe-b425182c6e92                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a8526b8d-8aa2-4914-af83-889cacfcc54e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a8526b8d-8aa2-4914-af83-889cacfcc54e
       Status: Executing Task...
    └── 🤖 Agent: Team Coordinator
            Status: In Progress

# Agent: Team Coordinator
## Task: Analyze this question and determine which experts need to be consulted: 'I need to calculate the volume of a sphere with radius r=5cm and then write a Python function to calculate this for any radius.'


🤖 Agent: Team Coordinator
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: a8526b8d-8aa2-4914-af83-889cacfcc54e
│      Assigned to: Team Coordinator
│      Status: ✅ Completed
│   ├── 🤖 Agent: Team Coordinator
│   │       Status: In Progress
│   └── 🤖 Agent: Team Coordinator
│           Status: ✅ Completed
├── 📋 Task: b02b24f5-3dba-4ab7-bf3c-662fca78b2b0
│      Assigned to: Programming Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Programming Expert
│           Status: ✅ Completed
├── 📋 Task: 277bf23c-6415-40eb-9bbb-b0b6ee0286dd
│      Assigned to: Mathematics Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Mathematics Expert
│           Status: ✅ Completed
├── 📋 Task: b7a3b923-da14-41a8-8502-cc7a81705c78
│      Assigned to: Physics Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Physics Expert
│           Status: ✅ Completed
└── 📋 Task: 3a574f37-622b-4729-8297-5eefaa6c1647
       Status: Executing Task...
    ├── 🤖 Agent: Team Coordinator
    │       Status: In Progress
    └── 🤖 Agent: Team Coordinator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: a8526b8d-8aa2-4914-af83-889cacfcc54e
│      Assigned to: Team Coordinator
│      Status: ✅ Completed
│   ├── 🤖 Agent: Team Coordinator
│   │       Status: In Progress
│   └── 🤖 Agent: Team Coordinator
│           Status: ✅ Completed
├── 📋 Task: b02b24f5-3dba-4ab7-bf3c-662fca78b2b0
│      Assigned to: Programming Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Programming Expert
│           Status: ✅ Completed
├── 📋 Task: 277bf23c-6415-40eb-9bbb-b0b6ee0286dd
│      Assigned to: Mathematics Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Mathematics Expert
│           Status: ✅ Completed
├── 📋 Task: b7a3b923-da14-41a8-8502-cc7a81705c78
│      Assigned to: Physics Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Physics Expert
│           Status: ✅ Completed
└── 📋 Task: 3a574f37-622b-4729-8297-5eefaa6c1647
       Assigned to: Team Coordinator
       Status: ✅ Completed
    ├── 🤖 Agent: Team Coordinator
    │       Status: In Progress
    └── 🤖 Agent: Team Coordinator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3a574f37-622b-4729-8297-5eefaa6c1647                                                                     │
│  Agent: Team Coordinator                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 34b9e3ae-c453-41a9-a1fe-b425182c6e92                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
To calculate the volume of a sphere, we use the mathematical formula:

\[ V = \frac{4}{3} \pi r^3 \]

where \( V \) is the volume and \( r \) is the radius of the sphere. In Python, we can implement this formula using the `math` library, which provides the constant `pi`.

Here is a Python function that calculates the volume of a sphere for any given radius:

```python
import math

def volume_of_sphere(radius):
    """
    Calculate the volume of a sphere given its radius.

    Parameters:
    radius (float): The radius of the sphere.

    Returns:
    float: The volume of the sphere.
    """
    return (4/3) * math.pi * (radius**3)

# Example usage:
radius = 5
volume = volume_of_sphere(radius)
print(f"The volume of a sphere with radius {radius} cm is approximately {volume:.2f} cm³")
```

### Explanation:

1. **Importing the `math` Library**: We import the `math` library to access the constant `pi`, which is crucial for our volume calculation.

2. **Defining the Function*

### 2. The Supervisor Pattern

In [5]:
from textwrap import dedent
from typing import List


# Create agents
supervisor = Agent(
    role="Project Supervisor",
    goal="Break down complex projects, manage tasks, and deliver comprehensive final reports",
    backstory=dedent("""
        You are an experienced project manager with a track record of successfully
        delivering complex projects. You excel at breaking down projects into manageable
        tasks, delegating effectively, and synthesizing results into clear final reports.
    """),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

researcher = Agent(
    role="Research Specialist",
    goal="Conduct thorough research on project requirements and background information",
    backstory=dedent("""
        You are a meticulous researcher with an eye for detail. You have a talent for
        finding relevant information quickly and synthesizing it into useful insights.
        Your research provides the foundation for successful project execution.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

analyst = Agent(
    role="Data Analyst",
    goal="Analyze project requirements and provide actionable insights",
    backstory=dedent("""
        You are a data analyst with expertise in interpreting requirements and identifying
        patterns. You can spot potential issues before they arise and suggest data-driven
        solutions to complex problems.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

implementer = Agent(
    role="Implementation Specialist",
    goal="Implement solutions based on project requirements and analysis",
    backstory=dedent("""
        You are a skilled implementer who can turn plans into reality. You have extensive
        experience in bringing concepts to life through practical and efficient solutions.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

tester = Agent(
    role="Quality Assurance Specialist",
    goal="Test implementations to ensure they meet project requirements",
    backstory=dedent("""
        You are a thorough QA specialist with an eye for detail. You can identify issues
        that others miss and ensure that implementations meet the highest quality standards.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

documenter = Agent(
    role="Documentation Specialist",
    goal="Create comprehensive documentation for project deliverables",
    backstory=dedent("""
        You are an expert in technical writing and documentation. You can turn complex
        technical information into clear, concise, and accessible documentation that serves
        as a valuable resource for all stakeholders.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Define tasks for a project
def create_project_tasks(project_description: str) -> List[Task]:
    """Create tasks for a project based on its description"""

    # Task breakdown task
    task_breakdown = Task(
        description=f"Break down this project into specific tasks: '{project_description}'. Identify key areas that need to be researched, analyzed, implemented, tested, and documented.",
        expected_output="A comprehensive task breakdown with specific requirements for each phase",
        agent=supervisor
    )

    # Research task
    research_task = Task(
        description=f"Research relevant information for: '{project_description}'. Use the task breakdown as a guide.",
        expected_output="A detailed research report with findings and recommendations",
        agent=researcher,
        context=[task_breakdown]
    )

    # Analysis task
    analysis_task = Task(
        description=f"Analyze the research findings and provide actionable insights for: '{project_description}'",
        expected_output="An analysis report with key insights and recommendations",
        agent=analyst,
        context=[task_breakdown, research_task]
    )

    # Implementation task
    implementation_task = Task(
        description=f"Create an implementation plan based on the analysis for: '{project_description}'",
        expected_output="A detailed implementation plan with specific steps",
        agent=implementer,
        context=[task_breakdown, research_task, analysis_task]
    )

    # Testing task
    testing_task = Task(
        description=f"Create a testing plan to ensure the implementation meets the requirements for: '{project_description}'",
        expected_output="A comprehensive testing plan with test cases",
        agent=tester,
        context=[task_breakdown, implementation_task]
    )

    # Documentation task
    documentation_task = Task(
        description=f"Create comprehensive documentation for: '{project_description}'",
        expected_output="Complete documentation covering all aspects of the project",
        agent=documenter,
        context=[task_breakdown, research_task, analysis_task, implementation_task, testing_task]
    )

    # Final report task
    final_report_task = Task(
        description=f"Create a final report synthesizing all findings and deliverables for: '{project_description}'",
        expected_output="A comprehensive final report that covers all aspects of the project",
        agent=supervisor,
        context=[research_task, analysis_task, implementation_task, testing_task, documentation_task]
    )

    return [task_breakdown, research_task, analysis_task, implementation_task, testing_task, documentation_task, final_report_task]

# Create the crew function
def execute_project(project_description: str):
    """Execute a project using the supervisor pattern"""

    # Create tasks for this specific project
    tasks = create_project_tasks(project_description)

    # Create the crew
    project_crew = Crew(
    agents=[supervisor, researcher, analyst, implementer, tester, documenter],
    tasks=tasks,
    verbose=True,  # Changed to True for detailed output or False for minimal output
    process=Process.sequential  # Tasks will be executed in the defined order
)

    # Get the crew to work!
    result = project_crew.kickoff()

    return result

# Example usage
if __name__ == "__main__":
    project_description = "Build a simple e-commerce website with product listings and a shopping cart"
    result = execute_project(project_description)
    print("\nFinal Project Report:")
    print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 912d9eb4-c0a8-4652-a17d-b8550c215023                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 94c9bce2-3e12-4e4d-8cf3-1f2a306dc70d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 94c9bce2-3e12-4e4d-8cf3-1f2a306dc70d
       Status: Executing Task...
    └── 🤖 Agent: Project Supervisor
            Status: In Progress

# Agent: Project Supervisor
## Task: Break down this project into specific tasks: 'Build a simple e-commerce website with product listings and a shopping cart'. Identify key areas that need to be researched, analyzed, implemented, tested, and documented.


🤖 Agent: Project Supervisor
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 94c9bce2-3e12-4e4d-8cf3-1f2a306dc70d
│      Assigned to: Project Supervisor
│      Status: ✅ Completed
│   ├── 🤖 Agent: Project Supervisor
│   │       Status: In Progress
│   ├── 🤖 Agent: Research Specialist
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Implementation Specialist
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Quality Assurance Specialist
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Project Supervisor
│           Status: ✅ Completed
├── 📋 Task: d67d1fe2-d15a-4096-ae9c-6c6e06250425
│      Assigned to: Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Specialist
│           Status: ✅ Completed
├── 📋 Task: 8dcfaf32-91da-41e2-a5f2-e535bc747759
│      Assigned to: Data Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Analyst
│           Status: ✅ Completed
├── 📋 Task: d8aebfd4-9137-43e3-93f1-00a2f7648052
│      Assigned to: Implementation Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Implementation Specialist
│           Status: ✅ Completed
├── 📋 Task: f39ca6ed-f63a-4490-b3af-cefa2c3c5f4e
│      Assigned to: Quality Assurance Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Quality Assurance Specialist
│           Status: ✅ Completed
├── 📋 Task: df7fd35f-2de6-46dd-9b2f-e93ede0cdc56
│      Assigned to: Documentation Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Documentation Specialist
│           Status: ✅ Completed
└── 📋 Task: 66617f59-350d-4ea1-8a0d-18423d83ce3b
       Status: Executing Task...
    ├── 🤖 Agent: Project Supervisor
    │       Status: In Progress
    └── 🤖 Agent: Project Supervisor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 94c9bce2-3e12-4e4d-8cf3-1f2a306dc70d
│      Assigned to: Project Supervisor
│      Status: ✅ Completed
│   ├── 🤖 Agent: Project Supervisor
│   │       Status: In Progress
│   ├── 🤖 Agent: Research Specialist
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Implementation Specialist
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Quality Assurance Specialist
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Project Supervisor
│           Status: ✅ Completed
├── 📋 Task: d67d1fe2-d15a-4096-ae9c-6c6e06250425
│      Assigned to: Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Specialist
│           Status: ✅ Completed
├── 📋 Task: 8dcfaf32-91da-41e2-a5f2-e535bc747759
│      Assigned to: Data Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Analyst
│           Status: ✅ Completed
├── 📋 Task: d8aebfd4-9137-43e3-93f1-00a2f7648052
│      Assigned to: Implementation Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Implementation Specialist
│           Status: ✅ Completed
├── 📋 Task: f39ca6ed-f63a-4490-b3af-cefa2c3c5f4e
│      Assigned to: Quality Assurance Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Quality Assurance Specialist
│           Status: ✅ Completed
├── 📋 Task: df7fd35f-2de6-46dd-9b2f-e93ede0cdc56
│      Assigned to: Documentation Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Documentation Specialist
│           Status: ✅ Completed
└── 📋 Task: 66617f59-350d-4ea1-8a0d-18423d83ce3b
       Assigned to: Project Supervisor
       Status: ✅ Completed
    ├── 🤖 Agent: Project Supervisor
    │       Status: In Progress
    └── 🤖 Agent: Project Supervisor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 66617f59-350d-4ea1-8a0d-18423d83ce3b                                                                     │
│  Agent: Project Supervisor                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 912d9eb4-c0a8-4652-a17d-b8550c215023                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Project Report:
# Final Report: Building a Simple E-Commerce Website with Product Listings and a Shopping Cart

## Introduction
This report provides a comprehensive overview of the project to build a simple e-commerce website with product listings and a shopping cart. It synthesizes findings from research, analysis, implementation, testing, and documentation to ensure a robust, scalable, and secure platform.

## Project Overview
The project involves creating an e-commerce platform that allows users to browse products, add them to a shopping cart, and complete purchases securely. The website is designed to be user-friendly, responsive, and secure, with a focus on providing a seamless shopping experience.

## Research Findings

### Front-End Technologies
- **HTML/CSS/JavaScript**: Essential for structuring, styling, and adding interactivity to the website.
- **React.js**: Recommended for building dynamic user interfaces.
- **Bootstrap**: Utilized for responsive and mobile-friendly

### 3. The Reflexion Pattern

In [6]:
from textwrap import dedent
from typing import List


# Define the OpenAI models
response_llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
reflection_llm = ChatOpenAI(model="gpt-4o", temperature=0)
revision_llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

# Create agents
responder = Agent(
    role="Response Generator",
    goal="Generate thoughtful and helpful responses to queries",
    backstory=dedent("""
        You are an expert at generating comprehensive and nuanced responses to complex questions.
        You have a deep understanding of various subject matters and can provide balanced,
        well-reasoned answers.
    """),
    verbose=True,
    allow_delegation=False,
    llm=response_llm
)

critic = Agent(
    role="Reflective Critic",
    goal="Critically analyze responses to identify weaknesses and areas for improvement",
    backstory=dedent("""
        You are a skilled critic with an eye for detail. You can identify logical fallacies,
        factual errors, biases, and other issues in responses. Your feedback is always
        specific, constructive, and actionable.
    """),
    verbose=True,
    allow_delegation=False,
    llm=reflection_llm
)

reviser = Agent(
    role="Response Reviser",
    goal="Improve responses based on critical feedback",
    backstory=dedent("""
        You are an expert at revising and improving content based on feedback. You can
        address identified issues while maintaining the strengths of the original response.
        Your revisions are clear, accurate, and comprehensive.
    """),
    verbose=True,
    allow_delegation=False,
    llm=revision_llm
)

evaluator = Agent(
    role="Quality Evaluator",
    goal="Evaluate if a response meets high quality standards",
    backstory=dedent("""
        You are a quality assurance specialist with a keen eye for excellence. You can
        determine whether a response meets high standards of quality, accuracy, and helpfulness.
        You make balanced judgments about when a response is ready for delivery.
    """),
    verbose=True,
    allow_delegation=False,
    llm=reflection_llm
)

finalizer = Agent(
    role="Response Finalizer",
    goal="Create polished final responses incorporating all feedback",
    backstory=dedent("""
        You specialize in creating final polished responses that incorporate all feedback
        and revisions. You ensure that the final response is cohesive, comprehensive, and
        addresses the original query effectively.
    """),
    verbose=True,
    allow_delegation=False,
    llm=response_llm
)

# Define tasks for the reflexion process
def create_reflexion_tasks(query: str, max_iterations: int = 3) -> List[Task]:
    """Create tasks for the reflexion process"""

    tasks = []

    # Initial response task
    initial_response_task = Task(
        description=f"Generate an initial comprehensive response to this query: '{query}'",
        expected_output="A thoughtful, well-reasoned initial response to the query",
        agent=responder
    )
    tasks.append(initial_response_task)

    # Create reflection-revision cycles
    for i in range(max_iterations):
        # Reflection task
        reflection_task = Task(
            description=f"Critically analyze the response to the query: '{query}'. Identify any issues, weaknesses, or areas for improvement.",
            expected_output="A detailed critique identifying specific issues and suggesting improvements",
            agent=critic,
            context=[initial_response_task] if i == 0 else [tasks[-1]]  # Use the previous task as context
        )
        tasks.append(reflection_task)

        # Revision task
        revision_task = Task(
            description=f"Revise the response to the query: '{query}' based on the critical feedback",
            expected_output="An improved response that addresses the issues identified in the feedback",
            agent=reviser,
            context=[tasks[-2], tasks[-1]]  # Use both the previous response and reflection
        )
        tasks.append(revision_task)

        # Evaluation task
        evaluation_task = Task(
            description=f"Evaluate whether the revised response to the query: '{query}' meets high quality standards or needs further improvement",
            expected_output="An evaluation determining whether the response is ready for finalization or needs another revision cycle",
            agent=evaluator,
            context=[tasks[-1]]  # Use the revised response
        )
        tasks.append(evaluation_task)

    # Final response task
    final_response_task = Task(
        description=f"Create a final polished response to the query: '{query}' that incorporates all feedback and revisions",
        expected_output="A polished final response that effectively addresses the original query",
        agent=finalizer,
        context=tasks  # Use all previous tasks as context
    )
    tasks.append(final_response_task)

    return tasks

# Create the crew function
def generate_reflective_response(query: str, max_iterations: int = 3):
    """Generate a response using the reflexion pattern"""

    # Create tasks for the reflexion process
    tasks = create_reflexion_tasks(query, max_iterations)

    # Create the crew
    reflexion_crew = Crew(
        agents=[responder, critic, reviser, evaluator, finalizer],
        tasks=tasks,
        verbose=True,  # Changed from 2 to True
        process=Process.sequential  # Tasks will be executed in the defined order
    )

    # Get the crew to work!
    result = reflexion_crew.kickoff()

    return result

# Example usage
if __name__ == "__main__":
    query = "What are the ethical implications of artificial general intelligence?"
    result = generate_reflective_response(query, max_iterations=2)
    print("\nFinal Response after Reflection:")
    print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 53349c95-1704-4fcb-8986-9ffebab0a29d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: edf2b83e-bd00-41ac-8e5f-da429df7ef77
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: edf2b83e-bd00-41ac-8e5f-da429df7ef77
       Status: Executing Task...
    └── 🤖 Agent: Response Generator
            Status: In Progress

# Agent: Response Generator
## Task: Generate an initial comprehensive response to this query: 'What are the ethical implications of artificial general intelligence?'


# Agent: Response Generator
## Final Answer: 
The ethical implications of artificial general intelligence (AGI) are profound and multifaceted, touching on numerous aspects of human society, morality, and the future of technology. Below are some of the key considerations:

1. **Autonomy and Control**: One of the primary ethical concerns is whether AGI will act autonomously beyond human control. The potential for AGI to make decisions without human input could lead to unintended consequences. Ensuring that AGI systems respect human values and can be controlled is critical.

2. **Moral Status**: If AGI achieves a level of consciousness or sentience, it raises questions about its moral status. Would AGI have rights? How should it be treated in comparison to humans and animals? These questions challenge current ethical and l

🚀 Crew: crew
└── 📋 Task: edf2b83e-bd00-41ac-8e5f-da429df7ef77
       Status: Executing Task...
    └── 🤖 Agent: Response Generator
            Status: ✅ Completed

...

🚀 Crew: crew
├── 📋 Task: edf2b83e-bd00-41ac-8e5f-da429df7ef77
│      Assigned to: Response Generator
│      Status: ✅ Completed
│   └── 🤖 Agent: Response Generator
│           Status: ✅ Completed
├── 📋 Task: 1b909d80-1f42-490b-ad02-b22ed3c86534
│      Assigned to: Reflective Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Reflective Critic
│           Status: ✅ Completed
├── 📋 Task: 0315b301-4180-459f-9530-5fd8dbb6c9e9
│      Assigned to: Response Reviser
│      Status: ✅ Completed
│   └── 🤖 Agent: Response Reviser
│           Status: ✅ Completed
├── 📋 Task: d67dbf80-3ca5-4a42-9915-e81239ab246e
│      Assigned to: Quality Evaluator
│      Status: ✅ Completed
│   └── 🤖 Agent: Quality Evaluator
│           Status: ✅ Completed
├── 📋 Task: 7d55c3d4-3a52-41b9-989d-b3283012052a
│      Assigned to: Reflective Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Reflective Critic
│           Status: ✅ Completed
├── 📋 Task: a385da23-47a1-4bf3-83b4-dcf65d683ffc
│      Assigned to: Response Reviser
│      Status: ✅ Completed
│   └── 🤖 Agent: Response Reviser
│           Status: ✅ Completed
├── 📋 Task: 6619fd21-ab95-4d98-9e47-44dc2bd2d5c3
│      Assigned to: Quality Evaluator
│      Status: ✅ Completed
│   └── 🤖 Agent: Quality Evaluator
│           Status: ✅ Completed
└── 📋 Task: 80cb6904-e0ae-4045-b403-34265e183074
       Status: Executing Task...
    └── 🤖 Agent: Response Finalizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: edf2b83e-bd00-41ac-8e5f-da429df7ef77
│      Assigned to: Response Generator
│      Status: ✅ Completed
│   └── 🤖 Agent: Response Generator
│           Status: ✅ Completed
├── 📋 Task: 1b909d80-1f42-490b-ad02-b22ed3c86534
│      Assigned to: Reflective Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Reflective Critic
│           Status: ✅ Completed
├── 📋 Task: 0315b301-4180-459f-9530-5fd8dbb6c9e9
│      Assigned to: Response Reviser
│      Status: ✅ Completed
│   └── 🤖 Agent: Response Reviser
│           Status: ✅ Completed
├── 📋 Task: d67dbf80-3ca5-4a42-9915-e81239ab246e
│      Assigned to: Quality Evaluator
│      Status: ✅ Completed
│   └── 🤖 Agent: Quality Evaluator
│           Status: ✅ Completed
├── 📋 Task: 7d55c3d4-3a52-41b9-989d-b3283012052a
│      Assigned to: Reflective Critic
│      Status: ✅ Completed
│   └── 🤖 Agent: Reflective Critic
│           Status: ✅ Completed
├── 📋 Task: a385da23-47a1-4bf3-83b4-dcf65d683ffc
│      Assigned to: Response Reviser
│      Status: ✅ Completed
│   └── 🤖 Agent: Response Reviser
│           Status: ✅ Completed
├── 📋 Task: 6619fd21-ab95-4d98-9e47-44dc2bd2d5c3
│      Assigned to: Quality Evaluator
│      Status: ✅ Completed
│   └── 🤖 Agent: Quality Evaluator
│           Status: ✅ Completed
└── 📋 Task: 80cb6904-e0ae-4045-b403-34265e183074
       Assigned to: Response Finalizer
       Status: ✅ Completed
    └── 🤖 Agent: Response Finalizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 80cb6904-e0ae-4045-b403-34265e183074                                                                     │
│  Agent: Response Finalizer                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 53349c95-1704-4fcb-8986-9ffebab0a29d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Response after Reflection:
The ethical implications of artificial general intelligence (AGI) are vast and complex, necessitating a comprehensive and nuanced analysis. The response to this query should address several critical areas to ensure a thorough understanding and exploration of the topic.

1. **Autonomy and Control**: One of the primary concerns with AGI is its potential to act autonomously beyond human control. The possibility that AGI could make decisions without human input poses risks of unintended consequences. To address these risks, mechanisms such as "kill switches," ethical alignment protocols, and continuous monitoring systems are critical. These strategies aim to ensure that AGI systems continue to respect human values and remain controllable.

2. **Moral Status**: If AGI achieves a level of consciousness or sentience, it raises significant questions about its moral status. Would AGI entities have rights, and how should they be treated in comparison to humans a

### 4. The Chain-of-Thought Pattern

In [7]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from textwrap import dedent
from typing import List

# Set environment variables
# Define the OpenAI models
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create agents
problem_analyzer = Agent(
    role="Problem Analyzer",
    goal="Break down complex problems into clear, logical steps for solution",
    backstory=dedent("""
        You are an expert at analyzing complex problems and breaking them down into
        well-defined steps. You can identify the key components of any problem and
        create a logical roadmap for solving it.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

step_reasoner = Agent(
    role="Step-by-Step Reasoner",
    goal="Execute reasoning steps with clear logic and thorough explanations",
    backstory=dedent("""
        You are a meticulous reasoning expert who excels at working through individual
        steps of complex problems. You show your work clearly, explain your reasoning
        thoroughly, and ensure each step builds logically on previous ones.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

solution_synthesizer = Agent(
    role="Solution Synthesizer",
    goal="Create clear, concise final answers based on complete reasoning chains",
    backstory=dedent("""
        You specialize in taking complex chains of reasoning and synthesizing them into
        clear, concise final answers. You can identify the key insights from a detailed
        reasoning process and express them in a way that directly addresses the original question.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Helper function to create tasks for a single reasoning step
def create_reasoning_step_task(problem: str, step_description: str, step_number: int, previous_steps=None) -> Task:
    context_str = ""
    if previous_steps:
        context_str = "Previous steps:\n" + "\n".join([f"Step {i+1}: {step}" for i, step in enumerate(previous_steps)])

    return Task(
        description=f"Execute reasoning step {step_number} for this problem:\n\n{problem}\n\n{context_str}\n\nCurrent step: {step_description}",
        expected_output=f"Detailed reasoning and solution for step {step_number}",
        agent=step_reasoner,
        context=[]  # Will be updated with previous tasks
    )

# Define the chain-of-thought process
def solve_with_chain_of_thought(problem: str):
    """Solve a problem using the chain-of-thought pattern"""

    # Task 1: Analyze the problem
    analysis_task = Task(
        description=f"Analyze this problem and break it down into a step-by-step plan for solution: '{problem}'",
        expected_output="A detailed plan with 3-7 clearly defined steps that will lead to the solution when followed sequentially",
        agent=problem_analyzer
    )

    # We need to execute the analysis task first to get the steps
    # In a real implementation, you would use CrewAI to get the result and then parse it
    # Here we'll simulate this with predefined steps for a math problem

    # Let's assume the analysis produces these steps (in a real implementation, this would come from the analysis_task)
    reasoning_steps = [
        "Understand what the variables represent and establish the relationship between upstream and downstream speed",
        "Set up equations using the relationship between distance, speed, and time for both parts of the journey",
        "Use the given information to find the distance traveled",
        "Solve for the speed of the current",
        "Verify the solution by checking if it makes sense in the context of the problem"
    ]

    # Create tasks for each reasoning step
    reasoning_tasks = []
    for i, step in enumerate(reasoning_steps):
        step_task = create_reasoning_step_task(
            problem=problem,
            step_description=step,
            step_number=i+1,
            previous_steps=[] if i == 0 else reasoning_steps[:i]
        )

        # Set the context for each step (the previous tasks)
        if i > 0:
            step_task.context = [analysis_task] + reasoning_tasks[:i]
        else:
            step_task.context = [analysis_task]

        reasoning_tasks.append(step_task)

    # Final task: Synthesize the solution
    synthesis_task = Task(
        description=f"Synthesize a clear final answer to this problem based on the complete reasoning chain: '{problem}'",
        expected_output="A concise, clear final answer that directly addresses the original problem",
        agent=solution_synthesizer,
        context=[analysis_task] + reasoning_tasks
    )

    # Create the full task list
    all_tasks = [analysis_task] + reasoning_tasks + [synthesis_task]

    # Create the crew
    cot_crew = Crew(
        agents=[problem_analyzer, step_reasoner, solution_synthesizer],
        tasks=all_tasks,
        verbose=True,  # Changed verbose from 2 to True for detailed output or False for minimal output
        process=Process.sequential  # Tasks will be executed in order
    )

    # Get the crew to work!
    result = cot_crew.kickoff()

    return result

# Example usage
if __name__ == "__main__":
    problem = "A boat travels upstream for 2 hours at 10 km/h and then returns downstream covering the same distance in 1 hour. What is the speed of the current?"
    result = solve_with_chain_of_thought(problem)
    print("\nSolution:")
    print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7534f4ba-d303-45e5-bcc5-8d2bbde01e17                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0a24a545-6f0b-4a5a-94f4-2d165279bc2d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0a24a545-6f0b-4a5a-94f4-2d165279bc2d
       Status: Executing Task...
    └── 🤖 Agent: Problem Analyzer
            Status: In Progress

# Agent: Problem Analyzer
## Task: Analyze this problem and break it down into a step-by-step plan for solution: 'A boat travels upstream for 2 hours at 10 km/h and then returns downstream covering the same distance in 1 hour. What is the speed of the current?'


# Agent: Problem Analyzer
## Final Answer: 
To solve the problem of finding the speed of the current when a boat travels upstream and downstream, we can break it down into the following steps:

1. **Define Variables:**
   - Let \( v_b \) be the speed of the boat in still water (in km/h).
   - Let \( v_c \) be the speed of the current (in km/h).

2. **Establish Equations for Upstream and Downstream:**
   - When the boat travels upstream, it moves against the current. Therefore, the effective speed of the boat upstream is \( v_b - v_c \).
   - When the boat travels downstream, it moves with the current. Therefore, the effective speed of the boat downstream is \( v_b + v_c \).

3. **Use Given Information to Formulate Equations:**

🚀 Crew: crew
└── 📋 Task: 0a24a545-6f0b-4a5a-94f4-2d165279bc2d
       Status: Executing Task...
    └── 🤖 Agent: Problem Analyzer
            Status: ✅ Completed

...

🚀 Crew: crew
├── 📋 Task: 0a24a545-6f0b-4a5a-94f4-2d165279bc2d
│      Assigned to: Problem Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Problem Analyzer
│           Status: ✅ Completed
├── 📋 Task: 93c5138e-399a-447c-8f97-6de531a861df
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: f73d877e-56b1-44d4-a832-4a10ad925f0d
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: bb020adb-2338-4cb9-9380-090284ee1585
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: 75a9f399-d618-4128-92dd-b625d81bb24f
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: 63226e38-8cba-4ca4-b3a7-6c910b736416
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
└── 📋 Task: 9533660d-f488-4183-9bbf-9d723fd4f852
       Status: Executing Task...
    └── 🤖 Agent: Solution Synthesizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0a24a545-6f0b-4a5a-94f4-2d165279bc2d
│      Assigned to: Problem Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Problem Analyzer
│           Status: ✅ Completed
├── 📋 Task: 93c5138e-399a-447c-8f97-6de531a861df
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: f73d877e-56b1-44d4-a832-4a10ad925f0d
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: bb020adb-2338-4cb9-9380-090284ee1585
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: 75a9f399-d618-4128-92dd-b625d81bb24f
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
├── 📋 Task: 63226e38-8cba-4ca4-b3a7-6c910b736416
│      Assigned to: Step-by-Step Reasoner
│      Status: ✅ Completed
│   └── 🤖 Agent: Step-by-Step Reasoner
│           Status: ✅ Completed
└── 📋 Task: 9533660d-f488-4183-9bbf-9d723fd4f852
       Assigned to: Solution Synthesizer
       Status: ✅ Completed
    └── 🤖 Agent: Solution Synthesizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9533660d-f488-4183-9bbf-9d723fd4f852                                                                     │
│  Agent: Solution Synthesizer                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7534f4ba-d303-45e5-bcc5-8d2bbde01e17                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Solution:
The speed of the current is approximately 3.33 km/h. This value ensures that the boat covers the same distance upstream and downstream, given the specified times and speeds.


### 5. RAG pattern

In [8]:
!pip install --upgrade crewai

In [12]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [13]:
# RAG Pattern Implementation with CrewAI
import os
from typing import List
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool  # Import BaseTool from crewai.tools
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Load environment variables
load_dotenv()

# Step 1: Set up the document ingestion process
def ingest_documents(docs_path: str, collection_name: str) -> Chroma:
    """Process documents and load them into a vector database."""
    # Simple document loading for demo purposes
    documents = []
    for filename in os.listdir(docs_path):
        if filename.endswith('.txt'):
            with open(os.path.join(docs_path, filename), 'r') as f:
                text = f.read()
                documents.append(Document(page_content=text, metadata={"source": filename}))

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(documents)

    # Create vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory="./chroma_db"
    )

    return vectorstore

# Define our retrieval tool using CrewAI's BaseTool
class RetrieveInformationTool(BaseTool):
    name: str = "retrieve_information"
    description: str = """
    Retrieve relevant information from the knowledge base based on the query.
    Use this tool when you need to search for specific information in the document database.
    """

    def _run(self, query: str, k: int = 3) -> str:
        """
        Args:
            query: The search query to find relevant documents
            k: Number of documents to retrieve (default: 3)
        Returns:
            A string containing the content of the retrieved documents and their sources
        """
        vectorstore = Chroma(
            persist_directory="./chroma_db",
            embedding_function=OpenAIEmbeddings()
        )
        retriever = vectorstore.as_retriever(search_kwargs={"k": k})
        docs = retriever.invoke(query)

        # Format the results
        if not docs:
            return "No relevant information found."

        results = []
        for i, doc in enumerate(docs, 1):
            source = doc.metadata.get("source", "Unknown")
            content = doc.page_content
            results.append(f"Document {i} (Source: {source}):\n{content}\n")

        return "\n".join(results)

# Define our CrewAI agents
def create_rag_crew(llm_model="gpt-4o"):
    # Create a shared LLM instance
    llm = ChatOpenAI(model=llm_model, temperature=0)

    # Create tool instances
    retrieve_tool = RetrieveInformationTool()

    # 1. Research Agent - responsible for retrieving information
    researcher = Agent(
        role="Research Expert",
        goal="Find the most relevant information from the knowledge base",
        backstory="""You're an expert researcher with a knack for finding the most relevant
        information to answer questions. You know how to query databases effectively and
        extract the key information.""",
        tools=[retrieve_tool],
        verbose=True,
        llm=llm
    )

    # 2. Analyzer Agent - formats and synthesizes the retrieved information
    analyzer = Agent(
        role="Information Analyst",
        goal="Process and organize retrieved information into a coherent context",
        backstory="""You excel at analyzing raw information and transforming it into a
        structured format that highlights the most important aspects. You can identify
        connections between different pieces of information.""",
        verbose=True,
        llm=llm
    )

    # 3. Writer Agent - generates the final answer
    writer = Agent(
        role="Expert Writer",
        goal="Create clear, accurate, and helpful responses based on the provided context",
        backstory="""You are a skilled writer who can craft informative and engaging responses.
        You focus on accuracy and clarity, making complex information easy to understand.""",
        verbose=True,
        llm=llm
    )

    # Define the tasks for our RAG process
    retrieval_task = Task(
        description="""
        Query the knowledge base to find relevant information to answer the question.
        Use the retrieve_information tool to search for up to 3 relevant documents.
        Return the full content of the retrieved documents along with their sources.

        The question is: {query}
        """,
        expected_output="""A detailed report of the retrieved information, including the
        full text content and sources of each document found.""",
        agent=researcher
    )

    analysis_task = Task(
        description="""
        Analyze the retrieved information and organize it into a coherent context.
        Identify the most relevant parts that address the question.
        Create a well-structured summary that will serve as context for generating the answer.

        Retrieved information: {retrieval_results}
        Question: {query}
        """,
        expected_output="""A concise, well-structured summary of the relevant information
        that addresses the question, with proper attribution to sources.""",
        agent=analyzer,
        context=[retrieval_task]
    )

    answer_task = Task(
        description="""
        Generate a comprehensive answer to the question using the provided context.
        Ensure your answer is accurate, helpful, and directly addresses the question.
        If the context doesn't contain sufficient information, acknowledge this and provide
        the best answer possible based on general knowledge.

        Context: {context}
        Question: {query}
        """,
        expected_output="""A clear, accurate, and helpful answer to the question that
        incorporates the relevant information from the context.""",
        agent=writer,
        context=[analysis_task]
    )

    # Create the crew
    crew = Crew(
        agents=[researcher, analyzer, writer],
        tasks=[retrieval_task, analysis_task, answer_task],
        process=Process.sequential,
        verbose=True
    )

    return crew

def main():
    # Set up your document collection (do this once)
    # ingest_documents("./documents", "my_knowledge_base")

    # Create the agents and tools
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    retrieve_tool = RetrieveInformationTool()

    researcher = Agent(
        role="Research Expert",
        goal="Find the most relevant information from the knowledge base",
        backstory="You're an expert researcher...",
        tools=[retrieve_tool],
        verbose=True,
        llm=llm
    )

    # Create a separate crew just for retrieval
    retrieval_task = Task(
        description="Query the knowledge base to find relevant information...",
        expected_output="A detailed report...",
        agent=researcher
    )

    retrieval_crew = Crew(
        agents=[researcher],
        tasks=[retrieval_task],
        verbose=True
    )

    # Run the retrieval crew first
    query = "What is the capital of France?"
    retrieval_results = retrieval_crew.kickoff(inputs={"query": query})

if __name__ == "__main__":
    main()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a50d550c-c557-41e2-a93d-821bf8b01ae6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6a9ce32b-a5a0-4ce2-8d7d-b174db0fd733
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6a9ce32b-a5a0-4ce2-8d7d-b174db0fd733
       Status: Executing Task...
    └── 🤖 Agent: Research Expert
            Status: In Progress

# Agent: Research Expert
## Task: Query the knowledge base to find relevant information...


🤖 Agent: Research Expert
    Status: In Progress

...



# Agent: Research Expert
## Final Answer: 
Unfortunately, I was unable to retrieve any relevant information from the knowledge base regarding the criteria for a detailed report. It seems that the specific information needed is not available in the current document database.




🚀 Crew: crew
└── 📋 Task: 6a9ce32b-a5a0-4ce2-8d7d-b174db0fd733
       Status: Executing Task...
    └── 🤖 Agent: Research Expert
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 6a9ce32b-a5a0-4ce2-8d7d-b174db0fd733
       Assigned to: Research Expert
       Status: ✅ Completed
    └── 🤖 Agent: Research Expert
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6a9ce32b-a5a0-4ce2-8d7d-b174db0fd733                                                                     │
│  Agent: Research Expert                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a50d550c-c557-41e2-a93d-821bf8b01ae6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯